Name: Soleil Umwiza

Student number: 4386019

I this notebook i am going to use the [sklearn Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) algorithm to classify emails as phishing or valid(safe). i will use a free and public dataset `phishing-and-ham-emails` downloaded from [Kaggle](https://www.kaggle.com/datasets/mohamedouledhamed/phishing-and-ham-emails). A phishing email can be recognized by a human by analyzing it's `Subject` and/or `Body text`. And since the subject and body are in unstructured free text format, Natural Language Processing with [NLTK](https://www.nltk.org/book/ch03.html) for `words tokenization` and `stemming` is used.

First, the versions of the required libraries are shown. It always wise to report the versions of the libraries used so that in case problems arise in the future, one can still go back to a state in which the notebook worked.

In [1]:
import re
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
import warnings

warnings.filterwarnings('ignore')



print("scikit-learn version:", sklearn.__version__)     # 1.1.3
print("pandas version:", pd.__version__)            # 1.5.1
print("seaborn version:", sns.__version__)   



scikit-learn version: 1.3.0
pandas version: 1.5.3
seaborn version: 0.12.2


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\csten\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# 📦 Data provisioning

Reading "phishing emails" and "valid emails" datasets

In [2]:
df1 = pd.read_csv("phishing.csv", index_col=0)
df2= pd.read_csv("data_extracted_easy_ham.csv", index_col=0)
print(df1.shape)
print(df2.shape)


(2239, 12)
(2551, 12)


# Exploring the data

## 📃 Sample the data

In [3]:
df1.sample(5)

,Fullname,To,Subject,Body,Date,IP,Return_Path,Delivered_to,Content_type,Message_Id,X_uid,Content_Length
From,,,,,,,,,,,,
admin@paypal.com,PayPal,user@example.com,Profile Update PayPal Inc. !,"Untitled Document Dear PayPal Inc. user, This ...","Wed, 28 Feb 2007 00:10:00 -0500 (EST)",NaN,test123@www-6.sitewizard.biz,user@example.com,text/html,1172639400.89692.qmail@,NaN,2579.0
technical-department_ref-368tg@53.com,53-rd Direct'2007,<umeet@uninet.edu>,The 53-rd Bank Direct Online Service Mail,"[, ]","Sat, 14 Apr 2007 08:50:55 -0400",216.208.126.16,technical-department_ref-368tg@53.com,user@login.example.com,"multipart/related; type=""multipart/alternative...",001901c77e93$8ae5078a$0414a8c0@FSTPC04,1123.0,NaN
operate-ref88585@53.com,Fifth Third Bank 2006 <operate-ref88585@53.com,user@example.com,Fifth Third Bank strongly recommends!,"[, ]","Tue, 21 Nov 2006 17:25:22 -0200",192.168.1.39,operate-ref88585@53.com,user@login.example.com,"multipart/mixed;\n boundary=""----------0E9DAD8...",322672305.20061121192522@53.com,451.0,NaN
customer_service@paypal.co,customer_service@paypal.co,undisclosed-recipients: ;,Notification of Limited Account Access (Routin...,Security Center Military Grade Encryption is O...,"Mon, 14 Jan 2002 15:41:55 -0600",86.127.28.54,user@mail.example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",20060215223855.20F8A52327C@macintrouble.net,302.0,NaN
aw-confirm@ebay.com,eBay,undisclosed-recipients: ;,Power Seller,"Dear eBay member, Congratulations! Your recent...","Sun, 4 Feb 2007 15:09:12 -0600",24.79.224.224,aw-confirm@ebay.com,user@login.example.com,"text/html;\n\tcharset=""Windows-1251""",SERVER1X4D9JlyB3G49000003ec@server1.triplettwe...,752.0,NaN


In [4]:
df2.sample(5)

,Fullname,To,Subject,Body,Date,IP,Return_Path,Delivered_to,Content_type,Message_Id,X_uid,Content_Length
From,,,,,,,,,,,,
eugen@leitl.org,Eugen Leitl <eugen@leitl.org,Lucas Gonze <lgonze@panix.com>,Re[2]: Selling Wedded Bliss (was Re: Ouch...),"On Mon, 9 Sep 2002, Lucas Gonze wrote:\n\n> a ...","Mon, 9 Sep 2002 16:23:57 +0200 (CEST)",NaN,fork-admin@xent.com,yyyy@localhost.example.com,TEXT/PLAIN; charset=US-ASCII,Pine.LNX.4.33.0209091613360.32400-100000@hydro...,NaN,NaN
rah@shipwright.com,R. A. Hettinga,"""Mr. FoRK"" <fork_list@hotmail.com>, <fork@exam...",Re: The Wrong Business,"At 10:27 PM -0700 on 9/30/02, Mr. FoRK wrote:\...","Tue, 1 Oct 2002 20:06:38 -0400",66.149.49.6,fork-admin@xent.com,yyyy@localhost.example.com,"text/plain; charset=""us-ascii""",p05111a35b9bfe33f9362@[66.149.49.6],NaN,NaN
rssfeeds@example.com,fark <rssfeeds@example.com,yyyy@example.com,Cat shot through eye with horse dart tranquili...,"URL: http://www.newsisfree.com/click/-1,841205...","Tue, 01 Oct 2002 08:01:11 -0000",127.0.0.1,rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210010801.g9181BK15568@dogma.slashnull.org,NaN,NaN
rssfeeds@example.com,scripting <rssfeeds@example.com,yyyy@example.com,"Jeremy Zawodny on life in Silicon Valley: ""I c...",URL: http://scriptingnews.userland.com/backiss...,"Thu, 26 Sep 2002 08:01:26 -0000",127.0.0.1,rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200209260801.g8Q81QC06426@dogma.slashnull.org,NaN,NaN
fork_list@hotmail.com,Mr. FoRK,"""Luis Villa"" <louie@ximian.com>, <fork@example...",Re: Slaughter in the Name of God,"----- Original Message -----\nFrom: ""Luis Vill...","Tue, 17 Sep 2002 13:40:12 -0700",NaN,fork-admin@xent.com,yyyy@localhost.example.com,"text/plain; charset=""iso-8859-1""",DAV26BcLJjiOivDY1FF00000bce@hotmail.com,NaN,NaN


 From the sample above, we can see that the `From` column is treated as index, however, we can see that the values are not unique per row. Hence, I will reset the index.

In [5]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [6]:
df1.head()

,From,Fullname,To,Subject,Body,Date,IP,Return_Path,Delivered_to,Content_type,Message_Id,X_uid,Content_Length
0,emailconfirm@ebay.com,emailconfirm@ebay.com,NaN,eBay account verification!,"Dear eBay User , After fraud complaints from t...","Thu, 30 Oct 2003 08:52:16 +0200",NaN,user@example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200502211804.NAA02450@mail.example.com,2.0,NaN
1,aw45-confirm@ebay.com,Ebay Team,undisclosed-recipients: ;,eBay - verify your account information,Welcome Welcome to a community of sellers that...,"Fri, 1 Jul 2005 17:57:54 -0700",217.129.34.6,user@mail.example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200507020500.BAA12022@mail.example.com,11.0,NaN
2,service@paypal.com,PayPal <service@paypal.com,user@example.com,Notification of limited account access,Dear valued PayPal® member: PayPal® is committ...,"Wed, 17 Aug 2005 18:28:02 +0000 (GMT)",NaN,user@mail.example.com,user@example.com,text/html,20050817182802.703BC84B0A0@asia.ibbmonitor.com,101.0,5004.0
3,service@intl.paypal.com,service@intl.paypal.com <service@intl.paypal.com,user@example.com,..PayPal Notification..: Update your information,PayPal You're Billing Information! Dear PayPal...,"Thu, 21 Sep 2006 04:46:36 +0200 (CEST)",NaN,wwwrun@s2.medienuniversum.com,user@login.example.com,text/html,20060921024636.9C7E3391672@s2.medienuniversum.com,295.0,8482.0
4,service@paypal.com,service@paypal.com,undisclosed-recipients:;,We recently noticed one or more attempts to lo...,We recently noticed one or more attempts to lo...,"Thu, 21 Sep 2006 05:52:53 +0200",193.253.255.81,service@paypal.com,user@login.example.com,"text/html;\n\tcharset=""Windows-1251""",20060921034817.322E41C00245@mwinf1921.orange.fr,296.0,1863.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2239 entries, 0 to 2238
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   From            2239 non-null   object 
 1   Fullname        2236 non-null   object 
 2   To              2234 non-null   object 
 3   Subject         2219 non-null   object 
 4   Body            2213 non-null   object 
 5   Date            2239 non-null   object 
 6   IP              1404 non-null   object 
 7   Return_Path     2239 non-null   object 
 8   Delivered_to    2239 non-null   object 
 9   Content_type    2213 non-null   object 
 10  Message_Id      2239 non-null   object 
 11  X_uid           1816 non-null   float64
 12  Content_Length  699 non-null    float64
dtypes: float64(2), object(11)
memory usage: 227.5+ KB


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551 entries, 0 to 2550
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   From            2551 non-null   object 
 1   Fullname        2535 non-null   object 
 2   To              2398 non-null   object 
 3   Subject         2551 non-null   object 
 4   Body            2551 non-null   object 
 5   Date            2551 non-null   object 
 6   IP              1703 non-null   object 
 7   Return_Path     2551 non-null   object 
 8   Delivered_to    2416 non-null   object 
 9   Content_type    2150 non-null   object 
 10  Message_Id      2551 non-null   object 
 11  X_uid           0 non-null      float64
 12  Content_Length  0 non-null      float64
dtypes: float64(2), object(11)
memory usage: 259.2+ KB


Looking into the data summary above, we have, we see that for some columns we have too many missing values. For example columns "Content_Length and X_uid" have too many null values. However, we need to be carefull before we can decide if we can drop them, because it could be that the value being null is a contributing factor that an email is `valid` or `phishing`.

Let's verify them carefully, by going back to the `phishing` and `valid` before they are merged.

Below we are checking the counts of the null values of the 2 columns `X_uid` and `Content_Length` for each of the datasets. We can see that in the `valid emails` the two columns have no values at all, and in the `phishing` dataset almost 75% of the Content_Length and almost 20% of X_uid values are missing.

In [9]:
print('Phishing dataframe: Count of null values in the column X_uid is {}/{} rows'.format(df1['X_uid'].isnull().sum(),len(df1.index)));
print('Phishing dataframe: Count of null values in the column Content_Length is {}/{} rows\n'.format(df1['Content_Length'].isnull().sum(),len(df1.index)));

print('Valid email dataframe: Count of null values in the column X_uid is {}/{} rows' .format(df2['X_uid'].isnull().sum(),len(df2.index)));
print('Valid email dataframe: Count of null values in the column Content_Length is {}/{} rows\n' .format(df2['Content_Length'].isnull().sum(),len(df2.index)));

print('Valid email dataframe: Count of null values in the column IP is {}/{} rows' .format(df2['IP'].isnull().sum(),len(df2.index)));
print('Valid email dataframe: Count of null values in the column IP is {}/{} rows' .format(df2['IP'].isnull().sum(),len(df2.index)));

Phishing dataframe: Count of null values in the column X_uid is 423/2239 rows
Phishing dataframe: Count of null values in the column Content_Length is 1540/2239 rows

Valid email dataframe: Count of null values in the column X_uid is 2551/2551 rows
Valid email dataframe: Count of null values in the column Content_Length is 2551/2551 rows

Valid email dataframe: Count of null values in the column IP is 848/2551 rows
Valid email dataframe: Count of null values in the column IP is 848/2551 rows


##### Now the `Content_length`, `IP` and `X_uid` columns can be dropped due to too many missing values and also their less relevance in this analysis

In [10]:
df1=df1.drop(columns=['IP','Content_Length','X_uid'])
df2=df2.drop(columns=['IP','Content_Length','X_uid'])


# 🛠️ Preprocessing


### 🎯 Target variable

Since we have two separate datasets one for phishing emails and one for valid emails, we will first label the data by adding a new column `IsEmailPhishing` on both datasets which will indicate if an email is phishing or not. For phishing emails we put `1` and for valid emails `0`


In [11]:
df1["IsEmailPhishing"] = 1

In [12]:
df1.IsEmailPhishing.value_counts()

1    2239
Name: IsEmailPhishing, dtype: int64

In [13]:
df2["IsEmailPhishing"] = 0


In [14]:
df2.IsEmailPhishing.value_counts()

0    2551
Name: IsEmailPhishing, dtype: int64

In [15]:
print(df1.shape)
print(df2.shape)

(2239, 11)
(2551, 11)


We will then do merging of both dataframes for further preprocessing and analysis

In [16]:
df = pd.concat([df1,df2], axis=0)
print(df.shape)

(4790, 11)


In [17]:
df.head(5)

,From,Fullname,To,Subject,Body,Date,Return_Path,Delivered_to,Content_type,Message_Id,IsEmailPhishing
0,emailconfirm@ebay.com,emailconfirm@ebay.com,NaN,eBay account verification!,"Dear eBay User , After fraud complaints from t...","Thu, 30 Oct 2003 08:52:16 +0200",user@example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200502211804.NAA02450@mail.example.com,1
1,aw45-confirm@ebay.com,Ebay Team,undisclosed-recipients: ;,eBay - verify your account information,Welcome Welcome to a community of sellers that...,"Fri, 1 Jul 2005 17:57:54 -0700",user@mail.example.com,user@example.com,"text/html;\n\tcharset=""Windows-1251""",200507020500.BAA12022@mail.example.com,1
2,service@paypal.com,PayPal <service@paypal.com,user@example.com,Notification of limited account access,Dear valued PayPal® member: PayPal® is committ...,"Wed, 17 Aug 2005 18:28:02 +0000 (GMT)",user@mail.example.com,user@example.com,text/html,20050817182802.703BC84B0A0@asia.ibbmonitor.com,1
3,service@intl.paypal.com,service@intl.paypal.com <service@intl.paypal.com,user@example.com,..PayPal Notification..: Update your information,PayPal You're Billing Information! Dear PayPal...,"Thu, 21 Sep 2006 04:46:36 +0200 (CEST)",wwwrun@s2.medienuniversum.com,user@login.example.com,text/html,20060921024636.9C7E3391672@s2.medienuniversum.com,1
4,service@paypal.com,service@paypal.com,undisclosed-recipients:;,We recently noticed one or more attempts to lo...,We recently noticed one or more attempts to lo...,"Thu, 21 Sep 2006 05:52:53 +0200",service@paypal.com,user@login.example.com,"text/html;\n\tcharset=""Windows-1251""",20060921034817.322E41C00245@mwinf1921.orange.fr,1


After merging the indices of the two merged datasets will be mixed up, meaning we will be duplicate index from `0` to `2239`. So, resetting the index is needed to have sequential index, then after getting a new index, we drop the mixed up one

In [18]:
df.reset_index(inplace=True)
df=df.drop(columns='index')

And we can verify that our new index is sequential by checking the last rows

In [19]:
df.tail()

,From,Fullname,To,Subject,Body,Date,Return_Path,Delivered_to,Content_type,Message_Id,IsEmailPhishing
4785,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Gene technique reveals human evolution,"URL: http://www.newsisfree.com/click/-3,871436...","Thu, 10 Oct 2002 08:04:25 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84PK14188@dogma.slashnull.org,0
4786,rssfeeds@example.com,guardian <rssfeeds@example.com,yyyy@example.com,Go-ahead for new-style hospitals,"URL: http://www.newsisfree.com/click/-4,872399...","Thu, 10 Oct 2002 08:04:10 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84AK14152@dogma.slashnull.org,0
4787,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Malicious code hidden in email software,"URL: http://www.newsisfree.com/click/-3,871080...","Thu, 10 Oct 2002 08:04:26 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84QK14191@dogma.slashnull.org,0
4788,rssfeeds@example.com,guardian <rssfeeds@example.com,yyyy@example.com,'Flexible' retirement gains ground,"URL: http://www.newsisfree.com/click/-4,872400...","Thu, 10 Oct 2002 08:04:10 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84AK14155@dogma.slashnull.org,0
4789,rssfeeds@example.com,newscientist <rssfeeds@example.com,yyyy@example.com,Critical US satellites could be hacked,"URL: http://www.newsisfree.com/click/-3,870882...","Thu, 10 Oct 2002 08:04:26 -0000",rssfeeds@example.com,yyyy@localhost.example.com,text/plain; encoding=utf-8,200210100804.g9A84QK14194@dogma.slashnull.org,0


Let's recheck the sumary of the whole dataset after merging 

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4790 entries, 0 to 4789
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   From             4790 non-null   object
 1   Fullname         4771 non-null   object
 2   To               4632 non-null   object
 3   Subject          4770 non-null   object
 4   Body             4764 non-null   object
 5   Date             4790 non-null   object
 6   Return_Path      4790 non-null   object
 7   Delivered_to     4655 non-null   object
 8   Content_type     4363 non-null   object
 9   Message_Id       4790 non-null   object
 10  IsEmailPhishing  4790 non-null   int64 
dtypes: int64(1), object(10)
memory usage: 411.8+ KB


### 💡 Feature selection

The column `To` and `Delivered_to` represent the recipients of the email, so these two columns are less relevent and will not have any added value on our analysis. The same apply to `Fullname`,`Message_Id`,`date` and `Return_Path` column.

The most relevant columns are `Subject`and `Body` since they contains words that will be tokenized and turned to features

In [21]:
df['Subject'].fillna('None', inplace=True)
df['Body'].fillna('None', inplace=True)

In [22]:
emailText_with_type=df[['Subject','Body','IsEmailPhishing']]

In [23]:
emailText_with_type.head()

,Subject,Body,IsEmailPhishing
0,eBay account verification!,"Dear eBay User , After fraud complaints from t...",1
1,eBay - verify your account information,Welcome Welcome to a community of sellers that...,1
2,Notification of limited account access,Dear valued PayPal® member: PayPal® is committ...,1
3,..PayPal Notification..: Update your information,PayPal You're Billing Information! Dear PayPal...,1
4,We recently noticed one or more attempts to lo...,We recently noticed one or more attempts to lo...,1


Merging Subject text with Body columns since both represent the text of an email

In [24]:
emailText_with_type['email_text']=df['Subject'] + ' ' + df['Body']

In [25]:
emailText_with_type.head()

,Subject,Body,IsEmailPhishing,email_text
0,eBay account verification!,"Dear eBay User , After fraud complaints from t...",1,"eBay account verification! Dear eBay User , Af..."
1,eBay - verify your account information,Welcome Welcome to a community of sellers that...,1,eBay - verify your account information Welcome...
2,Notification of limited account access,Dear valued PayPal® member: PayPal® is committ...,1,Notification of limited account access Dear va...
3,..PayPal Notification..: Update your information,PayPal You're Billing Information! Dear PayPal...,1,..PayPal Notification..: Update your informati...
4,We recently noticed one or more attempts to lo...,We recently noticed one or more attempts to lo...,1,We recently noticed one or more attempts to lo...


##### Tokenization and stemming

For this Iteration zero, I will only consider the columns `Body` and `Subject` for the analysis.
For that I will perform tokenization together with Stemming of the email text using NLTK pre processing. Within the function `text_processing`, I am using regex library to remove all ponctuations. Ref.[removing-stop-words-nltk-python](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/)


In [26]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [27]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

stop_words = set(stopwords.words('english'))
 
# extracting only sequence of letters and numbers from the text, converts the words in word_tokens to lower case and then checks whether
#they are present in stop_words or not, return tokens without stopwords in a list of words
def text_preprocessing(dataset):
    tokens_list=[]
    for i in range(0,len(dataset)):
        text=re.sub('[^a-zA-Z0-9]',' ',dataset['email_text'][i])
        text=text.lower()
        text=text.split()
        text=[ps.stem(w) for w in text if not w in stop_words]
        text =' '.join(text)
        tokens_list.append(text)
    return tokens_list

Taking a subset of the dataframe with only `email_text`and `IsEmailPhishing` so that I can start tokenization and preparing data for my Model

In [28]:
emailText_with_type=emailText_with_type[['email_text','IsEmailPhishing']]

In [29]:
emailText_with_type

,email_text,IsEmailPhishing
0,"eBay account verification! Dear eBay User , Af...",1
1,eBay - verify your account information Welcome...,1
2,Notification of limited account access Dear va...,1
3,..PayPal Notification..: Update your informati...,1
4,We recently noticed one or more attempts to lo...,1
...,...,...
4785,Gene technique reveals human evolution URL: ht...,0
4786,Go-ahead for new-style hospitals URL: http://w...,0
4787,Malicious code hidden in email software URL: h...,0
4788,'Flexible' retirement gains ground URL: http:/...,0


### Text processing using NLTK

In [30]:
email_words=text_preprocessing(emailText_with_type)

In [31]:
email_words

['ebay account verif dear ebay user fraud complaint ebay member ebay inc develop secur program fraudulend attempt account theft securis member inform updat check registr inform pleas updat inform complet form forward link check account valid ident login ebay order updat inform process take 5 day period abl acc ebay account period receiv instruct enter securis ebay account pleas click link sign account http signin ebay com aw cgi ebayisapi dll signin sspagenam h h sin us outlin user agreement ebay period send inform site chang enhanc visit privaci polici user agreement question regard safeharbor depart ebay inc',
 'ebay verifi account inform welcom welcom commun seller achiev except level success posit feedback ebay invit join us powersel agre rank pleas within 24 hour click highlight field day becom ebay power seller import registr activ becom powersel powersel ebay top seller sustain consist high volum monthli sale high level total feedback 98 posit better seller rank among success se

In [32]:
emailText_with_type['email_words']=email_words

Now since our dataset is ordered starting with phishing emails and ends with valid emails. To avoid that splitting our dataset may be unbalanced, I will shuffle the dataset using the `sample` function and save the shuffled dataset in a new variable. Then will reoder the columns so we have the target variable as a last column.

In [33]:
emailText_with_type= emailText_with_type.sample(frac=1)

In [34]:
emailText_with_type=emailText_with_type.reset_index(drop=True)

In [35]:
emailText_with_type

,email_text,IsEmailPhishing,email_words
0,"Re: From James> This is not an exmh problem, b...",0,jame exmh problem interact sendmail jame solar...
1,None None,1,none none
2,Re: Oh ma gawd - funny site of the day... Ahhh...,0,oh gawd funni site day ahhhh dave winer seem l...
3,eBay Account Issue - Response Required eBay se...,1,ebay account issu respons requir ebay sent mes...
4,Restore Your Account Access - user@example.com [],1,restor account access user exampl com
...,...,...,...
4785,Sheila Lennon was interviewed for the Times pi...,0,sheila lennon interview time piec url http scr...
4786,The Semantic Web makes me sick URL: http://div...,0,semant web make sick url http diveintomark org...
4787,Re[2]: Selling Wedded Bliss (was Re: Ouch...) ...,0,2 sell wed bliss ouch fri 6 sep 2002 eugen lei...
4788,Fifth Third Bank: account investigation warnin...,1,fifth third bank account investig warn


Changing order of the columns making the target column last.
Ref:[change-the-order-of-dataframe-columns](https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns)

In [36]:

cols = emailText_with_type.columns.tolist()
cols = cols[-1:] + cols[:-1]
emailText_with_type=emailText_with_type[cols]

In [37]:
emailText_with_type=emailText_with_type.drop(columns='email_text')

In [38]:
emailText_with_type.head()

,email_words,IsEmailPhishing
0,jame exmh problem interact sendmail jame solar...,0
1,none none,1
2,oh gawd funni site day ahhhh dave winer seem l...,0
3,ebay account issu respons requir ebay sent mes...,1
4,restor account access user exampl com,1


Assigning feature and target to variables

In [39]:

X = emailText_with_type['email_words']
y = emailText_with_type['IsEmailPhishing']

### 🪓 Splitting into train/test

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
print("There are in total", len(X), "observations, of which", len(X_train), "are now in the train set, and", len(X_test), "in the test set.")

There are in total 4790 observations, of which 3832 are now in the train set, and 958 in the test set.


In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500,ngram_range=(1,2))
X_train=cv.fit_transform(X_train).toarray()

In [42]:
X_test=cv.transform(X_test).toarray()

# 🧬 Modelling
In this step only the train set is used to fit the model, which in this case uses a Random forest Classifier algorithm for classification named [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). And after that the test set is used to calculate the model's accuracy, in other words how well it performs. Accuracy is a fraction where any value closer to 1 is considered better, and 1 itself (100% accurate) is usually impossible.
Ref: https://www.kaggle.com/code/prashantrajkaggle/spam-message-classification-with-nlp/notebook
     https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [43]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier().fit(X_train,y_train)

In [44]:
y_pred=classifier.predict(X_test)
                              

In [45]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

accuracy_score(y_test,y_pred)

0.9968684759916493

In [46]:
confusion_matrix(y_test,y_pred)

array([[484,   1],
       [  2, 471]], dtype=int64)

# 🔬 Evaluation
In order to shed some light on the results a classification report can be printed. This shows for every one of the classes how well the model performed.

In [47]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       485
           1       1.00      1.00      1.00       473

    accuracy                           1.00       958
   macro avg       1.00      1.00      1.00       958
weighted avg       1.00      1.00      1.00       958

